In [1]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
higgs_whole = pd.read_csv('./Predictions & Whole Data/HIGGS.csv', header=None, skiprows = range(0,10500000))

In [3]:
higgs_whole.shape

(500000, 29)

In [4]:
higgs_whole = higgs_whole[0].astype(int)
higgs_whole

0         0
1         1
2         1
3         0
4         0
         ..
499995    1
499996    1
499997    1
499998    0
499999    0
Name: 0, Length: 500000, dtype: int32

In [5]:
test_set = higgs_whole
default_preds = pd.read_csv('./Predictions & Whole Data/default_xgboost_preds_on_big_test_set.csv', header=None, dtype=int)
tuned_preds = pd.read_csv('./Predictions & Whole Data/optimized_xgboost_preds_on_big_test_set.csv', header=None, dtype=int)

In [6]:
mc_data = pd.DataFrame()
mc_data['ground_truth'] = higgs_whole
mc_data['y_pred_default'] = default_preds.values
mc_data['y_pred_tuned'] = tuned_preds.values
mc_data.head()

,ground_truth,y_pred_default,y_pred_tuned
0,0,0,0
1,1,1,1
2,1,1,1
3,0,0,0
4,0,0,0


In [7]:
mc_data.loc[mc_data['ground_truth']==mc_data['y_pred_default'],'correct_default']="Yes"
mc_data.loc[mc_data['ground_truth']==mc_data['y_pred_tuned'],'correct_tuned']="Yes"
mc_data.loc[mc_data['ground_truth']!=mc_data['y_pred_default'],'correct_default']="No"
mc_data.loc[mc_data['ground_truth']!=mc_data['y_pred_tuned'],'correct_tuned']="No"
mc_data.head()

,ground_truth,y_pred_default,y_pred_tuned,correct_default,correct_tuned
0,0,0,0,Yes,Yes
1,1,1,1,Yes,Yes
2,1,1,1,Yes,Yes
3,0,0,0,Yes,Yes
4,0,0,0,Yes,Yes


In [8]:
nr_corr_default_corr_tuned=0
nr_corr_default_incorr_tuned=0
nr_incorr_default_corr_tuned=0
nr_incorr_default_incorr_tuned=0

for index, row in mc_data.iterrows():
    
    if  row['correct_default']== "Yes" and  row['correct_tuned']=="Yes":
        nr_corr_default_corr_tuned += 1
        
    elif row['correct_default']== "Yes" and  row['correct_tuned']=="No":
        nr_corr_default_incorr_tuned += 1
        
    elif row['correct_default']== "No" and  row['correct_tuned']=="Yes":
        nr_incorr_default_corr_tuned += 1
        
    elif row['correct_default']== "No" and  row['correct_tuned']=="No":
        nr_incorr_default_incorr_tuned += 1

In [9]:
contingency_table_df=pd.DataFrame(data={"nr_correct_default":["Yes/Yes","No/Yes"], 
                                        "nr_incorrect_default":["Yes/No","No/No"]}, 
                                  index=["nr_correct_tuned","nr_incorrect_tuned"])
contingency_table_df

,nr_correct_default,nr_incorrect_default
nr_correct_tuned,Yes/Yes,Yes/No
nr_incorrect_tuned,No/Yes,No/No


In [10]:
contingency_table_df.iloc[0,0] = nr_corr_default_corr_tuned
contingency_table_df.iloc[0,1] = nr_incorr_default_corr_tuned
contingency_table_df.iloc[1,0] = nr_corr_default_incorr_tuned
contingency_table_df.iloc[1,1] = nr_incorr_default_incorr_tuned

contingency_table_df

,nr_correct_default,nr_incorrect_default
nr_correct_tuned,358807,17866
nr_incorrect_tuned,12205,111122


In [11]:
contingency_table = [[358807, 17866],
                     [12205, 111122]]

# Apply McNemar test
result = mcnemar(contingency_table, exact=False, correction=True)
print('Test statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
print()
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
    print('Same proportions of errors for the used models (fail to reject H0). Therefore the performance difference between two models are not statistically significant.')

else:
    print("""Different proportions of errors for the used models (reject H0). 
Therefore the performance difference between two models are statistically significant.""")

Test statistic=1065.332, p-value=0.000

Different proportions of errors for the used models (reject H0). 
Therefore the performance difference between two models are statistically significant.
